# Практика 1

---

Общие правила:

* Каждый день помимо лекции будет 1 практика - это `jupyter`-ноутбук, в котором есть задачки на разное количество правил.
* Рекомендуем выполнять задания в ноутбуке последовательно - они могут быть связаны друг с другом
* Выполнение не обязательно, но точно поможет лучше понять тему
* После выполнения практики (даже не целиком) следует скинуть ее проверяющим в телеграме в личку - либо Мише, либо Владу, как удобно)
* Также практика - отличный повод обсудить какие-то интересующие вещи (но если будут другие вопросы - все равно пишите).

---

На этой практике мы посмотрим на пример временного ряда, построим модели из лекции и получим первые приближения наших предсказаний!

Для начала склонируем наш репозиторий - там лежат нужные нам данные:

In [ ]:
!git clone https://github.com/ArChanDDD/TS-MCSSirius-2024.git

## Шаг 1: Знакомство с Pandas (1 балл)

В наше время, почти в любой ML-задаче нужно работать с _датасетами_ - структурированным набором данных. По большому счеты, это может быть обычный `.txt`-файл, но, само собой, работать с ним может быть не очень удобно.

Поэтому существуют библиотеки, которые предоставляют удобный функционал для работы с датасетами. `Pandas` - наверное, самая популярная такая библиотека.

На этом шаге мы познакомимся с этой библиотекой, посмотрим на функционал, и научимся выполнять базовые действия с датасетом.

In [ ]:
# Импортируем ее
import pandas as pd
# А эта библиотека для красивых графиков
import matplotlib.pyplot as plt
# А это чтобы крутые операции быстро делать
import numpy as np

Все это время мы будем работать со датасетом `DailyDelhiClimate` - он содержит информацию о погодных условиях в Дели в 2013-2017 годах.

Сначала стоит посмотреть, что там вообще есть:

In [ ]:
# Операция номер 0 -- чтение датасета.
df = pd.read_csv('TS-MCSSirius-2024/data/DailyDelhiClimateTrain.csv')

# Выводим только первые 5 строк:
df.head(5)

Понимать `pandas`-датасеты нужно так:
* Строчки - это записи
* Столбцы - это параметры, _фичи_

Получается, из параметров у нас есть

* Дата
* Средняя температура
* Влажность
* Скорость ветра
* Среднее давление

Например: 1 января 2013 года средняя температура была 10 градусов, влажность - 84.5, ветра не было, среднее давление - 1015 мм.рт.ст.

---

А теперь давайте разберемся, какие операции с датасетом можно делать (хотя корректнее его называть _ДатаФрейм_, но как удобно):

In [ ]:
# Cложение столбцов
df['meantemp'] + df['humidity']

Возвращает пандасовскую структуру - если хотим работать с ней как с чем-то более привычным, сделаем так:

In [ ]:
# Cложение столбцов, и список на выход
list(df['meantemp'] + df['humidity'])

и вот это уже обычный список - а с ним можно спокойно работать :)

Можно кстати создать новый столбец из списка:

In [ ]:
df['meantemp + humidity'] = list(df['meantemp'] + df['humidity'])
df

Здорово, да?

Помимо этого, столбцы аналогично поддерживают вычитание, деление, умножение, а также можно делать все эти действия с константой:

In [ ]:
# [:5] чтобы вернуть только первые 5 значений
(df['meantemp'] * 2)[:5]

Давай чуть-чуть попрактикуемся - посчитай для первых пяти записей разность удвоенной температуры и скорости ветра:

### Задачка 1 (0.5 балла)

In [ ]:
# TODO

result = ...

assert len(result) == 5, "Нужно посчитать только для первых 5 записей"
assert np.allclose(list(result), [20.0, 11.82, 9.7, 16.1, 8.3]), "Что-то посчитано не так"

Иногда появляется необходимость итерироваться по записям в датасете - и, к сожалению, в пандасе с этим все грустно.
Но есть иной вариант:

In [ ]:
# Обернем датасет в массив из numpy
np.array(df[:5])

Как видно, теперь это обычный список из списков, каждый из которых соответствует одной записи. А по списку итерироваться уже не так и сложно!

### Задачка 2 (0.5 балла)

Следующая задачка - посчитать сумму всех температур с помощью цикла `for`.

In [ ]:
# TODO

result = ...

assert np.isclose(result, 37274.45)

Но вообще эту задачу можно было решить и так:

In [ ]:
result_2 = sum(df['meantemp'])

print(f'Было  - {result}')
print(f'Стало - {result_2}')

# Шаг 2: Обзор данных

Раз уж мы там в датасет напихали всякого, давайте заново его импортируем



In [ ]:
df = pd.read_csv('TS-MCSSirius-2024/data/DailyDelhiClimateTrain.csv')
df.head()

На самом деле тут есть проблемка, которую мы сейчас пофиксим:

In [ ]:
df['date']

Как мы видим, тип стоит `object` - это плохо, потому что графики будут кривыми (можете построить, если интересно). Но это фиксится просто:

In [ ]:
from datetime import datetime

df['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in list(df['date'])]

df['date']

Теперь хорошо!

**Наша главная цель** - научиться предсказывать температуру, поэтому, чтобы не забыть, создадим столбец `target`, который будет равен столбцу `meantemp`.

In [ ]:
df['target'] = df['meantemp']

А теперь давайте вообще посмотрим на наш временной ряд:

In [ ]:
_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df['date'], df['target']) # это значит, что по оси ОХ будет дата, а по оси OY будет наша температура

Первое правило при работе с любой задачей машинного обучения - понять, что за данные. Что мы можем сказать про этот ряд? Есть ли у него тренд? А периодичность? Можно его назвать стационарным?

# Шаг 3: Наивная модель (3 балла)

Понимаю, хочется начать предсказывать - мы же тут как раз для этого!

Поэтому, давайте начнем с базовой модели - наивной. На всякий случай напомню, что это:

> **Наивной моделью** для задачи прогнозирования временных рядов называется модель, которая сопоставляет $y_t$ значение, которое было $lag$ шагов назад - то есть $y_{t-lag}$

> Для оценки наших моделей мы будем использовать валидацию на последнем "горбу" - вот этой четвертой горке на графике.

> Для этого мы разобьем датасет на две части - на одной мы будем обучаться, а на другой проверять качество наших моделек. Такие датасеты принято называть `train` и `val` -- для обучения и валидации соответственно.

In [ ]:
# Валидация начнется с 1117 значения
val_start = 1117

df_train = df[:val_start]
df_val = df[val_start:]

_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df_train['date'], df_train['target'], label='train')
_ = plt.plot(df_val['date'], df_val['target'], label='val')
_ = plt.legend()

In [ ]:
# Просто функция для построения графиков, можете не обращать много внимания)
def plot_graph(y_pred):
  _, _ = plt.subplots(1,1,figsize=(18,10))
  _ = plt.plot(df_train['date'], df_train['target'], label='train')
  _ = plt.plot(df_val['date'], df_val['target'], label='val')
  _ = plt.plot(df_val['date'], y_pred, label='predict')
  _ = plt.legend()

### Задача 3 - реализация наивной модели (1.5 балла)

In [ ]:
lag = len(df_val) # пусть будет так - но если хочется, можно другое число поставить

### TODO ###
# Необходимо для каждого i < len(df_val) добавить в y_pred то значение, которое было в df_train lag позиций назад
# То есть для lag=5 у нас получится так:
# * train = [1,2,3,4,5,6,7]
# * pred  = [3,4,5]

y_pred = []

for i in range(len(df_val)):
  ...


plot_graph(y_pred)

Вначале вроде как даже неплохо, но потом становится совсем плохо...

Давайте оценим насколько все плохо - для этого посчитаем функцию
$$
MSE = \frac{1}{n}\sum_{i=0}^n (y_i - \hat{y}_i)^2
$$

Где:

* $y_i$ - правильно значение (которое `val`)
* $\hat{y}_i$ - предсказанное значение (которое `y_pred`)
* $n$ - длина массива `y_pred` и `val`

Реализуйте эту функцию!

### Задача 4 - реализация MSE (1.5 балла)

In [ ]:
# TODO

def MSE(y_true, y_pred):
  result = ...
  return result

In [ ]:
mse = MSE(list(df_val['target']), y_pred)
print(mse)

assert np.isclose(mse, 17.575336), "Функция делает что-то не так..."

Сложно сказать, хорошо это или нет, но будем считать эту ошибку ориентиром - _бейзлайном_. Наша цель - сделать ее как можно меньше (а значит предсказание станет наиболее точным).

# Шаг 4: Линейная регрессия (6 баллов)

Наш следующий шаг - построить линейную регрессию. К огромному счастью, в питоне это все уже есть, а значит наша задача лишь подготовить данные

## Регрессия на одном параметре (2 балла)

Для начала давайте попробуем, как в примере на лекции, построить регрессию, в которой `x` будет равен дню в месяце.

<details><summary> Подсказка 1 </summary>

Не забывайте о том, что по `df_train['date']` можно итерироваться!

</details>

<details><summary> Подсказка 2 </summary>

Для каждого `x` в `df_train['date']` можно сделать `x.day` и получить день месяца.

</details>

In [ ]:
# Достаем модель - линейную регрессию
from sklearn.linear_model import LinearRegression

# Почти у всех основных моделей в ML есть основные методы:
# * fit() - обучение модели, на вход как правило подается X - данные, y - целевое значение
# * predict() - предсказание, на вход подается X - возвраещается y_pred

# Определяем модель
model = LinearRegression()

# TODO
# X и X_val строятся одинаковым алгоритмом, но на разных датасетах. Мы чуть раньше рассказывали чем они отличаются.

y = list(df_train['target'])
X = ...
X_val = ...


# Тут сначала оборачиваем в массив numpy, потом делаем так, чтобы числа были в столбик, а не в строку
X = np.array(X).reshape(-1, 1)
X_val = np.array(X_val).reshape(-1, 1)
# Ну и обучаем, конечно
model.fit(X, y)

y_pred = model.predict(X_val)

plot_graph(y_pred)

Получилось что-то странное, не так ли? И в этом на самом деле нет ничего странного - насколько мы помним, наша функция принимала вид $y=kx$, а даже при учете, что дни чередуются, описать что-то хорошо не получится :(

В любом случае, посчитаем MSE

In [ ]:
MSE(y_pred, list(df_val['target']))

Cурово. Попробуем сделать регрессию на нескольких параметрах - построим $y=\sum k_ix_i$

## Регрессия на нескольких параметрах (4 балла)

Вообще, по графику можно увидеть, что линейной функцией приблизить не получится - вот бы можно было сделать что-то вроде $y=k_1x_1 + k_2x_2^2$...

Но ведь так можно сделать! С одно стороны, это уже полиномиальная регрессия, с другой стороны - это линейная регрессия, в которой есть полиномы. Поэтому давайте создадим эти полиномы - сделайте аналогично прыдущему пункту, но теперь мы будем обучаться не на `[day]`, а на `[day, day^2, month, month^2]`

In [ ]:
model = LinearRegression()

# TODO

y = list(df_train['target'])
X = ...
X_val = ...


# Тут сначала оборачиваем в массив numpy, потом делаем так, чтобы числа были в столбик, а не в строку
X = np.array(X).reshape(-1, 4)
X_val = np.array(X_val).reshape(-1,4)
# Ну и обучаем, конечно
model.fit(X, y)

y_pred = model.predict(X_val)

plot_graph(y_pred)

In [ ]:
MSE(y_pred, list(df_val['target']))

Ну это ведь уже совсем другое дело! Даже на графике видно, что стало заметно лучше, а MSE стал лучше почти на 40%! И все благодаря тому, что мы добавили несколько экзогенных переменных.

На самом деле, Линейная регрессия - штука очень мощная. И не просто так всякие дата-саентисты часто ей пользуются - ведь она умеет так:

In [ ]:
list(model.coef_)

Это коэфициенты нашей модели - $i$-й коэфициент соответсвует $i$-му параметру в $X$.

Видно, что модель обращает большое внимание на месяц, а не на день - именно поэтому наша предыдущая модель была такой плохой.

---

Не думайте, что линейная регрессия это не тру - ради интереса можете попробовать добавить еще параметров. Вот увидите, результат может быть очень крутым 😁